In [1]:
!nvidia-smi

Thu Feb 29 08:47:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:08:00.0 Off |                    0 |
|  0%   27C    P8    21W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch

torch.__version__

'2.1.2+cu118'

In [2]:
# Installing necessary libraries

!pip install transformers datasets evaluate scikit-learn
!pip install accelerate -U
!pip install pandas polars pickle5 tabulate
!pip install wandb tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.0
    Uninstalling accelerate-0.27.0:
      Successfully uninstalled accelerate-0.27.0
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Checking if system is running on cuda

if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

print(device)

cuda:0


In [4]:
# Importing libraries 
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)
from transformers import logging
import torch
import pandas as pd

logging.set_verbosity_error()# Suppress warning

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Choosing tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.generation_config.pad_token_id = model.generation_config.eos_token_id

In [5]:
# Prompts and decoding algorithms
prompts = [
    "Today I believe we can finally",
    "In a world full of chaos",
    "The future of technology will bring",
    "When the sun sets",
    "Once upon a time"
]
decoding_algorithms = [
    ("Greedy Search", {"do_sample":False, "max_length": 30}),
    ("Beam Search", {"num_beams": 5, "max_length": 30, "early_stopping": True}),
    ("Top-K Sampling", {"do_sample": True, "max_length": 30, "top_k": 50}),
    ("Top-P Sampling", {"do_sample": True, "max_length": 30, "top_p": 0.75})]
     

In [6]:
# Function to calculate perplexity
def calculate_perplexity(max_length, stride, input_ids):
    seq_len = input_ids.size(1)

    # max_length = 2 #max length of context
    # stride = 1

    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl

In [7]:
# Output text and perplexity for each method
results = []

for prompt in prompts:
    row = {"Prompt": prompt}
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    for algorithm_name, params in decoding_algorithms:
        outputs = model.generate(input_ids, **params)
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        output_ids = tokenizer(generated_text, return_tensors="pt").input_ids
        perplexity = calculate_perplexity(2,1,output_ids)
        row[algorithm_name] = generated_text
        row[f"{algorithm_name} Parameters"] = params
        row[f"{algorithm_name} Perplexity"] = perplexity
    results.append(row)

df = pd.DataFrame(results)

/home/yaoyi/pyo00005/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you ma

In [9]:
# Make spreadsheet
df.to_csv('./task1_output.csv')